In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
import re

In [2]:
cd

C:\Users\37789


In [3]:
df = pd.read_csv("Final.csv")

In [4]:
df

,Airline,Source,Destination,Departure,Arrival,Number of Stops,Class,Date,Stopover_1_Time,Stopover_1_Airport,Stopover_2_Time,Stopover_2_Airport,Stopover_3_Time,Stopover_3_Airport,Total_Stopover_Time,Operated,price in CAD,date,days_left
0,Etihad Airways,Toronto Pearson Intl,Bengaluru Intl,10:10 pm,3:05 am+2,1 stop,Economy Comfort,2024-06-02,2h 10m,Abu Dhabi Zayed Intl,NaN,NaN,NaN,NaN,2h 10m,NaN,2340,2024-06-02,1
1,Delta,Toronto Pearson Intl,Bengaluru Intl,10:40 pm,11:15 pm+2,1 stop,Basic Economy,2024-06-02,22h 15m,Amsterdam Schiphol,NaN,NaN,NaN,NaN,22h 15m,KLM,1347,2024-06-02,1
2,"Lufthansa, Vistara",Toronto Pearson Intl,Bengaluru Intl,9:20 pm,7:45 am+2,2 stops,Economy Basic,2024-06-02,2h 05m,Frankfurt am Main,4h 55m,Mumbai Chhatrapati Shivaji Intl,NaN,NaN,7h 0m,Air Canada,1934,2024-06-02,1
3,"Lufthansa, Vistara",Toronto Pearson Intl,Bengaluru Intl,9:20 pm,6:50 am+2,2 stops,Economy Basic,2024-06-02,1h 55m,Frankfurt am Main,4h 25m,Hyderabad Rajiv Gandhi Intl,NaN,NaN,6h 20m,Air Canada,2291,2024-06-02,1
4,"Air Canada, Air India",Toronto Pearson Intl,Bengaluru Intl,11:40 pm,6:25 am+2,2 stops,Flex,2024-06-02,1h 25m,London Heathrow,1h 30m,New Delhi Indira Gandhi Intl,NaN,NaN,2h 55m,NaN,2661,2024-06-02,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73814,American Airlines,Sao Paulo Guarulhos Intl,Toronto Pearson Intl,10:00 pm,4:24 pm+1,1 stop,Flagship Business,2024-07-21,6h 04m,Dallas/Fort Worth,NaN,NaN,NaN,NaN,6h 4m,NaN,4032,2024-07-21,50
73815,United Airlines,Sao Paulo Guarulhos Intl,Toronto Pearson Intl,9:00 pm,2:52 pm+1,2 stops,Business,2024-07-21,3h 20m,Newark,1h 25m,Chicago O'Hare Intl,NaN,NaN,4h 45m,Republic Airways DBA United Express,4004,2024-07-21,50
73816,United Airlines,Sao Paulo Guarulhos Intl,Toronto Pearson Intl,9:00 pm,2:52 pm+1,2 stops,Business,2024-07-21,2h 19m,Newark,2h 26m,Chicago O'Hare Intl,NaN,NaN,4h 45m,Republic Airways DBA United Express,4004,2024-07-21,50
73817,United Airlines,Sao Paulo Guarulhos Intl,Toronto Pearson Intl,9:50 pm,3:43 pm+1,2 stops,Business,2024-07-21,1h 40m,Houston George Bush Intcntl,2h 52m,Chicago O'Hare Intl,NaN,NaN,4h 32m,NaN,4004,2024-07-21,50


In [5]:
# Function to extract extra days and clean arrival time
def extract_days(arrival):
    match = re.search(r'\+(\d+)', arrival)
    if match:
        extra_days = int(match.group(1))
        arrival_time = re.sub(r'\+\d+', '', arrival).strip()
    else:
        extra_days = 0
        arrival_time = arrival.strip()
    return arrival_time, extra_days

# Apply the function to the Arrival column and expand the results into two new columns
df[['CleanArrival', 'ExtraDays']] = df['Arrival'].apply(lambda x: pd.Series(extract_days(x)))

# Update the Arrival column with the cleaned time
df['Arrival'] = df['CleanArrival']

# Drop the CleanArrival column as it is no longer needed
df.drop('CleanArrival', axis=1, inplace=True)

In [6]:
df.isnull().sum()

Airline                    0
Source                     0
Destination               15
Departure                  0
Arrival                    0
Number of Stops            0
Class                      0
Date                       0
Stopover_1_Time          495
Stopover_1_Airport       495
Stopover_2_Time        22866
Stopover_2_Airport     22866
Stopover_3_Time        65669
Stopover_3_Airport     65669
Total_Stopover_Time        0
Operated               40299
price in CAD               0
date                       0
days_left                  0
ExtraDays                  0
dtype: int64

In [7]:
# Drop the specified columns
df.drop(['Stopover_3_Time', 'Stopover_3_Airport', 'Operated'], axis=1, inplace=True)

In [8]:
df['date'] = pd.to_datetime(df['date'], format='%Y-%m-%d')

# Extract day, month, and year into separate columns
df["Day"] = df["date"].dt.day
df["Month"] = df["date"].dt.month
df["Year"] = df["date"].dt.year

# Drop the original 'Date_of_journey' column
df.drop(columns=["date"], inplace=True)
df.drop(columns=["Date"], inplace=True)

In [9]:
# Assuming df is your DataFrame and these are the columns you want to encode
columns_to_encode = ['Departure', 'Arrival', 'Airline','Destination','Source']

# Initialize LabelEncoder
le = LabelEncoder()

# Encode each column
for column in columns_to_encode:
    df[column + '_encoded'] = le.fit_transform(df[column])
    
df.drop(columns=columns_to_encode, inplace=True)

In [10]:
df["Number of Stops"] = df["Number of Stops"].replace('direct', '0')

df["Number of Stops"] = df["Number of Stops"].str.replace(r" stops?", "", regex=True)

df["Number of Stops"] = pd.to_numeric(df["Number of Stops"])

In [11]:
economy_classes = [
    'Economy Comfort', 'Basic Economy', 'Economy Basic', 'Standard', 'Economy Flex', 
    'Economy Fully Flex', 'Economy Deluxe', 'Economy Light', 'Light', 'Economy Saver', 
    'Basic', 'Eco Special', 'Economy', 'Economy Lite', 'Economy Standard', 
    'Economy Super Saver', 'Discount', 'Promotion', 'Plus', 'Economy Convenience', 
    'Standard Economy', 'Economy (fully refundable)', 'Econo', 'Economy Plus', 
    'AM Plus', 'Economy Classic', 'Saver', 'Economy Basic Plus', 'Economy Value', 
    'Semi-Flexible', 'Elight', 'Super Saver', 'Economy Full Flex', 'Economy Flexi', 
    'Main Cabin', 'Refundable Main', 'Economy Select Pro', 'Economy Flexi', 
    'Main Cabin Semi-Flex', 'LowFare'
]

premium_economy_classes = [
    'Premium Economy Fully Flex', 'Premium Flexible', 'Prem Economy', 'Premium Lowest', 
    'Premium Economy Essential', 'Premium Economy Flex', 'Premium Economy Basic', 
    'Premium Economy Fully Flex', 'Premium Economy Standard', 'Best', 'Premium', 
    'Premium Economy', 'Premium Select', 'Premium Standard', 'Premium (fully refundable)', 
    'Premium Lite', 'Premium Full Flex', 'Premium Economy Basic Plus', 
    'Premium (partially refundable)', 'Premium Flex', 'Eco Flex', 'Eco Flex Plus', 
    'Premium Semi Flex', 'Refundable Premium', 'Premium Economy Optima', 
    'Premium Economy Flexible', 'Premium Economy Basic Plus'
]

business_classes = [
    'Flex', 'Comfort', 'Main Cabin Flex', 'Latitude', 'Business Lowest', 'Business Comfort', 
    'Business Deluxe', 'Business Value', 'Business Flexible', 'Business Flex', 
    'Flagship Business Flexible', 'Business Fully Flex', 'Delta One', 'Business', 
    'Business Semi Flex', 'Business Standard', 'Business Basic', 'Mixed', 'Business Saver', 
    'Business Semi-Flexible', 'Business Promotion', 'Business Platinum', 'Business Special', 
    'Business Essential', 'Executive', 'Business Lite', 'Saga Premium', 'Business/First', 
    'Business Basic Plus', 'Business Leisure', 'Business/First (fully refundable)', 
    'Best Buy', 'Best Offer', 'Business Optima', 'DISCOUNT', 'Flagship Business', 
    'Business Light', 'Business Elite', 'Business Promotional', 'Flexible', 'Business Flex Plus', 
    'BusinessFlex', 'Premium Business Plus', 'Business Promo', 'Full', 'Promo', 
    'Business Classic', 'PorterReserve Navigate', 'Business Flexi', 'AM Plus Flex', 
    'Business Full Flex', 'Economy Flexi', 'Refundable Delta One', 'Business Classic', 
    'Good Deal', 'Business Full Flex', 'Business Flexi', 'Main Cabin Semi-Flex', 
    'Business Flex Plus', 'Saver', 'Business Promo', 'Business Flexi', 'Business Classic'
]

def categorize_class(value):
    if value in economy_classes:
        return 'Economy'
    elif value in premium_economy_classes:
        return 'Premium Economy'
    elif value in business_classes:
        return 'Business'
    else:
        return 'Economy' 
df["Class"] = df["Class"].apply(categorize_class)

In [12]:
df['Economy'] = df['Class'].apply(lambda x: 1 if x == 'Economy' else 0)
df['Business'] = df['Class'].apply(lambda x: 1 if x == 'Business' else 0)
df['Premium Economy'] = df['Class'].apply(lambda x: 1 if x == 'Premium Economy' else 0)

df.drop('Class', axis=1, inplace=True)

In [13]:
# Function to fill null values with mode
def fill_null_with_mode(df, column_name):
    mode_value = df[column_name].mode()[0]
    df[column_name].fillna(mode_value, inplace=True)

# Fill missing values using mode for each column
fill_null_with_mode(df, 'Stopover_1_Time')
fill_null_with_mode(df, 'Stopover_1_Airport')
fill_null_with_mode(df, 'Stopover_2_Time')
fill_null_with_mode(df, 'Stopover_2_Airport')

In [14]:
# Assuming df is your DataFrame and these are the columns you want to encode
columns_to_encode = ["Stopover_1_Airport","Stopover_2_Airport"]

# Initialize LabelEncoder
le = LabelEncoder()

# Encode each column
for column in columns_to_encode:
    df[column + '_encoded'] = le.fit_transform(df[column])
    
df.drop(columns=columns_to_encode, inplace=True)

In [15]:
# Function to convert time to total minutes and round off
def convert_time_to_minutes(time_str):
    hours = re.search(r'(\d+)h', time_str)
    minutes = re.search(r'(\d+)m', time_str)
    hours = int(hours.group(1)) if hours else 0
    minutes = int(minutes.group(1)) if minutes else 0
    total_minutes = hours * 60 + minutes
    return round(total_minutes, 2)  # Round to 2 decimal places

# Function to convert time to total hours and round off
def convert_time_to_hours(time_str):
    hours = re.search(r'(\d+)h', time_str)
    minutes = re.search(r'(\d+)m', time_str)
    hours = int(hours.group(1)) if hours else 0
    minutes = int(minutes.group(1)) if minutes else 0
    total_hours = hours + minutes / 60
    return round(total_hours, 2)  # Round to 2 decimal places

# Apply the conversion functions to the DataFrame columns for Stopover_1_Time
df['Stopover_1_Time_Minutes'] = df['Stopover_1_Time'].apply(convert_time_to_minutes)
df['Stopover_1_Time_Hours'] = df['Stopover_1_Time'].apply(convert_time_to_hours)

# Apply the conversion functions to the DataFrame columns for Stopover_2_Time
df['Stopover_2_Time_Minutes'] = df['Stopover_2_Time'].apply(convert_time_to_minutes)
df['Stopover_2_Time_Hours'] = df['Stopover_2_Time'].apply(convert_time_to_hours)

# Apply the conversion function to the DataFrame column for Total_Stopover_Time
df['Total_Stopover_Time_Minutes'] = df['Total_Stopover_Time'].apply(convert_time_to_minutes)
df['Total_Stopover_Time_Hours'] = df['Total_Stopover_Time'].apply(convert_time_to_hours)

In [16]:
df = df.drop(columns=['Stopover_1_Time', 'Stopover_2_Time', 'Total_Stopover_Time'])

In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 73819 entries, 0 to 73818
Data columns (total 23 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Number of Stops              73819 non-null  int64  
 1   price in CAD                 73819 non-null  int64  
 2   days_left                    73819 non-null  int64  
 3   ExtraDays                    73819 non-null  int64  
 4   Day                          73819 non-null  int32  
 5   Month                        73819 non-null  int32  
 6   Year                         73819 non-null  int32  
 7   Departure_encoded            73819 non-null  int32  
 8   Arrival_encoded              73819 non-null  int32  
 9   Airline_encoded              73819 non-null  int32  
 10  Destination_encoded          73819 non-null  int32  
 11  Source_encoded               73819 non-null  int32  
 12  Economy                      73819 non-null  int64  
 13  Business        

In [18]:
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [19]:
y = df['price in CAD']
X = df.drop(columns=['price in CAD'])

In [20]:
X = sm.add_constant(X)

In [21]:
model = sm.OLS(y, X).fit()

In [22]:
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:           price in CAD   R-squared:                       0.405
Model:                            OLS   Adj. R-squared:                  0.405
Method:                 Least Squares   F-statistic:                     2641.
Date:                Thu, 13 Jun 2024   Prob (F-statistic):               0.00
Time:                        00:47:37   Log-Likelihood:            -6.5615e+05
No. Observations:               73819   AIC:                         1.312e+06
Df Residuals:                   73799   BIC:                         1.313e+06
Df Model:                          19                                         
Covariance Type:            nonrobust                                         
                                  coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------
Number of Stops   

In [23]:
vif_data = pd.DataFrame()
vif_data["Feature"] = X.columns
vif_data["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
print(vif_data)

C:\Users\37789\anaconda3\Lib\site-packages\statsmodels\stats\outliers_influence.py:198: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)
C:\Users\37789\anaconda3\Lib\site-packages\statsmodels\regression\linear_model.py:1781: RuntimeWarning: divide by zero encountered in scalar divide
  return 1 - self.ssr/self.centered_tss


                        Feature           VIF
0               Number of Stops  4.891116e+00
1                     days_left           inf
2                     ExtraDays  2.264552e+00
3                           Day           inf
4                         Month           inf
5                          Year  0.000000e+00
6             Departure_encoded  1.074081e+00
7               Arrival_encoded  1.048071e+00
8               Airline_encoded  1.136261e+00
9           Destination_encoded  2.517145e+00
10               Source_encoded  2.022875e+00
11                      Economy           inf
12                     Business           inf
13              Premium Economy           inf
14   Stopover_1_Airport_encoded  1.053955e+00
15   Stopover_2_Airport_encoded  1.186866e+00
16      Stopover_1_Time_Minutes  1.704202e+06
17        Stopover_1_Time_Hours  1.704079e+06
18      Stopover_2_Time_Minutes  1.329642e+06
19        Stopover_2_Time_Hours  1.329674e+06
20  Total_Stopover_Time_Minutes  2

In [24]:
# Extract p-values
p_values = model.pvalues

In [25]:
threshold = 0.05
features_above_threshold = p_values[p_values > threshold]
print("Features with p-value greater than 0.05:")
print(features_above_threshold)

Features with p-value greater than 0.05:
Departure_encoded    0.051423
dtype: float64


In [26]:
X = X.drop(columns=['Departure_encoded'])

In [27]:
# Add a constant term to the feature matrix
X = sm.add_constant(X)
model = sm.OLS(y, X).fit()

In [28]:
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:           price in CAD   R-squared:                       0.405
Model:                            OLS   Adj. R-squared:                  0.405
Method:                 Least Squares   F-statistic:                     2787.
Date:                Thu, 13 Jun 2024   Prob (F-statistic):               0.00
Time:                        00:49:21   Log-Likelihood:            -6.5615e+05
No. Observations:               73819   AIC:                         1.312e+06
Df Residuals:                   73800   BIC:                         1.313e+06
Df Model:                          18                                         
Covariance Type:            nonrobust                                         
                                  coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------
Number of Stops   